# Analisis Financiero del las 500 Empresas que componen el S&P500

In [2]:
# TODO: Importación de librerias y utilidades

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
sns.set()

In [3]:
# TODO: Extracción de tabla con las principales características de las empresas que componene el S&P500

wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
wiki_sp500 = wiki[0]
wiki_sp500

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [4]:
# TODO: función arreglar anomalias

x = lambda x : x.replace('.', '-')
wiki_sp500['Symbol'] = wiki_sp500['Symbol'].apply(x)

In [5]:
# TODO: List de nombres del SP500

tickers = list(wiki_sp500['Symbol'])

In [6]:
# TODO: Traer datos de las Acciones de yahoo finance api

raw_data = yf.download(tickers=tickers, start='2000-01-01', end='2021-12-31', interval='1d', threads=True, group_by='ticker')

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- FCX: No data found for this date range, symbol may be delisted
- CEG: Data doesn't exist for startDate = 946702800, endDate = 1640926800


In [7]:
# TODO: Convertir tabla a Single Index

df_original = raw_data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)
df_original.reset_index()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2000-01-03,A,44.143154,51.502148,56.464592,48.193848,56.330471,4674353.0
1,2000-01-03,AAPL,0.854541,0.999442,1.004464,0.907924,0.936384,535796800.0
2,2000-01-03,ABC,3.016693,3.890625,3.937500,3.812500,3.875000,2784800.0
3,2000-01-03,ABMD,18.250000,18.250000,18.656250,18.250000,18.515625,185600.0
4,2000-01-03,ABT,9.306480,15.711531,16.160433,15.599306,15.823756,10635087.0
...,...,...,...,...,...,...,...,...
2472615,2021-12-30,YUM,137.071228,138.389999,139.149994,138.199997,138.699997,538500.0
2472616,2021-12-30,ZBH,124.015045,124.533981,125.631065,124.339806,124.631065,1304701.0
2472617,2021-12-30,ZBRA,597.539978,597.539978,605.489990,596.159973,602.710022,87500.0
2472618,2021-12-30,ZION,62.372440,63.130001,64.120003,63.070000,63.450001,413100.0


In [8]:
# TODO: Obtener una copia del dataframe original

df = df_original.copy().sort_values(['Date', 'Ticker'])
df.reset_index(inplace=True)


In [9]:
# TODO: Informacion general del dataframe

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2472620 entries, 0 to 2472619
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   Date       datetime64[ns]
 1   Ticker     object        
 2   Adj Close  float64       
 3   Close      float64       
 4   High       float64       
 5   Low        float64       
 6   Open       float64       
 7   Volume     float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 150.9+ MB


In [10]:
# TODO: Cantidad de valores nulos por columna

df.isna().sum()

Date         0
Ticker       0
Adj Close    0
Close        0
High         0
Low          0
Open         0
Volume       0
dtype: int64

In [11]:
# TODO: Agregar dia de la semana al dataframe

days = {
    0:'Mon',
    1:'Tues',
    2:'Weds',
    3:'Thurs',
    4:'Fri',
    5:'Sat',
    6:'Sun'
}

df['Week_Day'] = df['Date'].dt.dayofweek
# df['Week_Day'] = df['Week_Day'].apply(lambda x: days[x])

In [12]:
# TODO: Separar en 500 dataframes para realizar los diferentes calculos recomendados

list_sp500 = df['Ticker'].unique()

list_df = []
for i in list_sp500:
    df_ticker = df[df['Ticker'] == i]
    df_ticker['GAP_Ret'] = np.log(df_ticker['Open'] / df_ticker['Close'].shift(1)).fillna(0)
    df_ticker['Intra_Ret'] = np.log(df_ticker['Close'] / df_ticker['Open']).fillna(0)
    df_ticker['Variations'] = (df_ticker['Adj Close'].pct_change()).fillna(0)
    df_ticker['Volatility'] = (df_ticker['Variations'].rolling(250).std()*100*(250)**0.5).fillna(0)
    list_df.append(df_ticker)

/tmp/ipykernel_669/198540076.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ticker['GAP_Ret'] = np.log(df_ticker['Open'] / df_ticker['Close'].shift(1)).fillna(0)
/tmp/ipykernel_669/198540076.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ticker['Intra_Ret'] = np.log(df_ticker['Close'] / df_ticker['Open']).fillna(0)
/tmp/ipykernel_669/198540076.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [13]:
# TODO: Concatener dataframes con los calculos en un solo dataframe

df_cal = pd.DataFrame()
for i in list_df:
    df_cal = pd.concat([df_cal, i], ignore_index=True)

In [14]:
# TODO: Agregar año de la fecha

df_cal['Year'] = pd.DatetimeIndex(df_cal['Date']).year

In [15]:
# TODO: Agregar Año y Fecha

df_cal['Year/Month'] = df_cal['Date'].dt.strftime('%Y-%m')

### Descripcion de los Datos

**Conclusiones:**
- Obtenemos un boxplot

### Cual es el mejor dia para invertir teniendo en cuenta el retorno de los movimientos GAP

In [16]:
df_days = df_cal.groupby('Week_Day', as_index=False).mean()
df_days['GAP_Ret']

0    0.000039
1    0.000577
2    0.000114
3    0.000055
4    0.000220
Name: GAP_Ret, dtype: float64

### Variaciones ADJ Variation por dia

### Cual es el mejor día para invertir teniendo en cuenta el retorno de los movimientos intradiarios

### Cuales son las mejores industrias que pertenecen al SP500 en las cuales se puede invertir


In [17]:
df_ind = pd.merge(df_cal, wiki_sp500[['Symbol', 'GICS Sector']], how='left', left_on='Ticker', right_on='Symbol')

In [18]:
df_ind

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Week_Day,GAP_Ret,Intra_Ret,Variations,Volatility,Year,Year/Month,Symbol,GICS Sector
0,2000-01-03,A,44.143154,51.502148,56.464592,48.193848,56.330471,4674353.0,0,0.000000,-0.089612,0.000000,0.0,2000,2000-01,A,Health Care
1,2000-01-04,A,40.771095,47.567955,49.266811,46.316166,48.730328,4765083.0,1,-0.055322,-0.024142,-0.076389,0.0,2000,2000-01,A,Health Care
2,2000-01-05,A,38.242085,44.617310,47.567955,43.141991,47.389126,5758642.0,2,-0.003767,-0.060271,-0.062030,0.0,2000,2000-01,A,Health Care
3,2000-01-06,A,36.785965,42.918453,44.349072,41.577251,44.080830,2534434.0,3,-0.012097,-0.026723,-0.038076,0.0,2000,2000-01,A,Health Care
4,2000-01-07,A,39.851467,46.494991,47.165592,42.203148,42.247852,2819626.0,4,-0.015748,0.095791,0.083333,0.0,2000,2000-01,A,Health Care
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472615,2021-12-23,OGN,29.742706,30.219999,30.379999,29.900000,30.000000,957700.0,3,-0.000333,0.007307,0.006998,0.0,2021,2021-12,OGN,Health Care
2472616,2021-12-27,OGN,29.614759,30.090000,30.280001,29.827000,30.170000,1250100.0,0,-0.001656,-0.002655,-0.004302,0.0,2021,2021-12,OGN,Health Care
2472617,2021-12-28,OGN,29.417919,29.889999,30.230000,29.850000,29.930000,1256700.0,1,-0.005332,-0.001337,-0.006647,0.0,2021,2021-12,OGN,Health Care
2472618,2021-12-29,OGN,29.349024,29.820000,29.980000,29.629999,29.879999,1309100.0,2,-0.000335,-0.002010,-0.002342,0.0,2021,2021-12,OGN,Health Care


In [19]:
df_ind['Var_Pct_Intra'] = (((df_ind['Close'] - df_ind['Open']) / df_ind['Open']) * 100)
df_ind['Var_Pct_Intra']


0          -8.571424
1          -2.385317
2          -5.849055
3          -2.636920
4          10.052911
             ...    
2472615     0.733331
2472616    -0.265164
2472617    -0.133648
2472618    -0.200801
2472619     2.638607
Name: Var_Pct_Intra, Length: 2472620, dtype: float64

In [20]:
df_ind_g = df_ind.groupby('GICS Sector', as_index=False).sum()

In [21]:
df_ind_g = df_ind_g[['GICS Sector', 'Var_Pct_Intra']].sort_values('Var_Pct_Intra', ascending=False)
df_ind_g

,GICS Sector,Var_Pct_Intra
7,Information Technology,17394.960109
5,Health Care,17218.799196
6,Industrials,15221.581020
4,Financials,11909.018832
1,Consumer Discretionary,9838.741014
2,Consumer Staples,8040.786400
9,Real Estate,7775.467006
10,Utilities,4952.423413
8,Materials,3244.312498
0,Communication Services,2541.541502


In [22]:
df_cal_sample = df_cal.sample(n=10000)
df_cal_sample

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Week_Day,GAP_Ret,Intra_Ret,Variations,Volatility,Year,Year/Month
38715,2021-11-17,ADI,184.562164,187.039993,188.100006,185.539993,188.029999,2242300.0,2,0.001916,-0.005279,-0.003357,26.490041,2021,2021-11
1955969,2008-06-02,WFC,18.743578,27.110001,27.540001,26.709999,27.469999,23179200.0,0,-0.003634,-0.013192,-0.016685,40.820186,2008,2008-06
203387,2016-05-31,AVY,66.669456,74.379997,74.779999,73.860001,74.750000,788500.0,1,0.001071,-0.004962,-0.003884,22.974596,2016,2016-05
759290,2003-12-18,FMC,5.992271,7.263660,7.283174,7.035993,7.068517,785885.0,3,0.000000,0.027233,0.027607,37.003013,2003,2003-12
1852209,2013-12-30,UNP,69.132217,83.025002,83.355003,82.665001,83.050003,1840400.0,0,0.000301,-0.000301,0.000000,17.331980,2013,2013-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809462,2005-05-20,GS,80.144463,101.080002,101.199997,99.550003,99.949997,3495500.0,4,-0.001699,0.011242,0.009588,18.202262,2005,2005-05
152228,2011-01-26,AON,40.293873,45.770000,45.919998,45.549999,45.619999,1905000.0,2,0.001316,0.003283,0.004610,20.028199,2011,2011-01
774201,2019-03-26,GD,152.701126,166.899994,168.500000,165.919998,167.919998,905900.0,1,0.007772,-0.006093,0.001681,22.090891,2019,2019-03
175182,2014-04-23,APH,22.063374,23.882500,23.987499,23.492500,23.975000,2694400.0,2,0.009324,-0.003866,0.005473,20.410873,2014,2014-04


### Cuales fueron los momentos de alta volatilidad que afectaron al SP500


In [23]:
vix = df_cal.groupby('Date', as_index=False).mean()
vix

,Date,Adj Close,Close,High,Low,Open,Volume,Week_Day,GAP_Ret,Intra_Ret,Variations,Volatility,Year
0,2000-01-03,21.265919,32.355345,33.471253,31.654053,33.163465,6.223430e+06,0.0,0.000000,-0.019114,0.000000,0.000000,2000.0
1,2000-01-04,20.535243,31.272141,32.424119,30.866489,31.850683,6.203550e+06,1.0,-0.011636,-0.015548,-0.026297,0.000000,2000.0
2,2000-01-05,20.646323,31.469527,32.139686,30.453542,31.219827,8.633296e+06,2.0,-0.001303,0.007144,0.006505,0.000000,2000.0
3,2000-01-06,20.539055,31.495979,32.397363,30.613203,31.381994,7.130797e+06,3.0,-0.002012,0.006766,0.005611,0.000000,2000.0
4,2000-01-07,21.273072,32.586286,32.987435,31.271572,31.746873,5.880348e+06,4.0,0.003860,0.021615,0.026491,0.000000,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5530,2021-12-23,196.239610,197.499064,199.095086,195.836442,196.920813,3.364791e+06,3.0,0.003619,0.002992,0.006681,27.730037,2021.0
5531,2021-12-27,198.994732,200.263776,200.776958,197.054927,198.127652,3.175022e+06,0.0,0.001494,0.011115,0.012796,27.751650,2021.0
5532,2021-12-28,198.997487,200.269824,202.067318,198.901423,200.467416,3.024613e+06,1.0,-0.000010,0.001389,0.001420,27.744181,2021.0
5533,2021-12-29,199.911513,201.187681,202.381321,199.274683,200.451012,2.846840e+06,2.0,0.000360,0.002492,0.002894,27.723788,2021.0


### Cuales son las 9 mejores empresas para invertir

In [24]:
df_tic = df_cal.groupby(['Ticker', 'Year'], as_index=False).mean()
df_tic

,Ticker,Year,Adj Close,Close,High,Low,Open,Volume,Week_Day,GAP_Ret,Intra_Ret,Variations,Volatility
0,A,2000,42.750335,49.877136,51.826004,48.017594,49.966151,4.238259e+06,2.031746,0.001448,-0.002890,0.000991,1.248551
1,A,2001,20.495584,23.912350,24.570816,23.174646,23.837856,3.311148e+06,2.016129,-0.006980,0.004349,-0.001709,85.102047
2,A,2002,14.069000,16.414407,16.814270,16.050480,16.433482,3.700359e+06,2.015873,-0.000934,-0.000900,-0.001093,57.393866
3,A,2003,12.409934,14.478762,14.686315,14.223693,14.447368,3.244672e+06,2.007937,0.000044,0.001890,0.002365,59.905769
4,A,2004,16.479940,19.227269,19.540103,18.952046,19.241320,4.267569e+06,2.019841,-0.000084,-0.000683,-0.000474,36.609528
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9890,ZTS,2017,59.254587,61.130040,61.508486,60.700239,61.065100,2.696791e+06,2.039841,0.000076,0.001107,0.001256,18.006718
9891,ZTS,2018,83.414196,85.541992,86.363227,84.728048,85.557450,2.503187e+06,2.023904,0.000817,-0.000132,0.000817,18.917575
9892,ZTS,2019,108.343593,110.373135,111.279087,109.404920,110.319841,2.109394e+06,2.015873,0.001104,0.000628,0.001843,22.630343
9893,ZTS,2020,142.945261,144.786205,146.621818,142.957194,144.740039,2.024208e+06,2.007905,0.000413,0.000471,0.001210,34.456841


In [25]:
df_tic = df_tic[['Ticker', 'GAP_Ret', 'Intra_Ret']].sort_values('GAP_Ret', ascending=False)
df_tic = df_tic[:9]
df_tic

,Ticker,GAP_Ret,Intra_Ret
3222,EQIX,0.018353,-0.020011
4029,GRMN,0.014814,0.000009
3221,EQIX,0.014506,-0.025421
3223,EQIX,0.014284,-0.025355
385,AIG,0.013191,-0.013375
5504,LVS,0.011318,-0.007652
7893,SBAC,0.011072,-0.024794
1533,C,0.010807,-0.013612
5535,LYV,0.010681,0.014278


In [26]:
df_tic = df_tic[['Ticker', 'GAP_Ret', 'Intra_Ret']].sort_values('Intra_Ret', ascending=False)
df_tic = df_tic[:9]
df_tic

,Ticker,GAP_Ret,Intra_Ret
5535,LYV,0.010681,0.014278
4029,GRMN,0.014814,0.000009
5504,LVS,0.011318,-0.007652
385,AIG,0.013191,-0.013375
1533,C,0.010807,-0.013612
3222,EQIX,0.018353,-0.020011
7893,SBAC,0.011072,-0.024794
3223,EQIX,0.014284,-0.025355
3221,EQIX,0.014506,-0.025421


In [27]:
df_Chg_Pct = (((df_cal['Close'].fillna(method='bfill').iloc[-1])-(df_cal['Open'].fillna(method='bfill').iloc[0]))/(df_cal['Open'].fillna(method='bfill').iloc[0]))*100